# Import continuous data from .continuous Openephys raw files

In [ ]:
""" Takes the .npy with raw data from openephys2npy, and create baseline and OF epochs 

Input: 
    'SERTXXXX/npys_dir + structure.npy' 
    
    Of npy matrices by structure, with 
        - Channels at 30 KHz
        - All channels with mean subtracted
        - Low-passed at 300 Hz
        - Median subtracted
        
Output:
    1. Epoched data from 3s before to after each center entrance. 
    2. Epoched baselines (mouse in OF periphery) of 3s lenght
    3. Epochs saved in '/home/maspe/filer/SERT/SERTXXXX/npys/mPFC_epochs.npy'
    
        
Baseline epochs are obtaines from:
    
    'SERT/ALL/npys/baselines.dict'
    
    * A dictionary with n baselines for each mouse
    * Sampled at 30 Hz from the start of the 10 min OF.
    
    
Written by Mauricio Aspé-Sánchez. Copyleft 2020.

""" 

#### Import the required modules

In [1]:
# Import required modules
#import glob
#import sys
import pickle
import numpy as np
import pandas as pd
import physig as ps
from scipy import signal
from matplotlib import pyplot as plt

#### Get the mouse name and create the file name and folder paths

In [2]:
IDs = ['SERT1597']
ID = 'SERT1597'

In [3]:
# Setting working file and paths
files_dir = '/home/maspe/filer/SERT/' + ID + '/continuous/'
npys_dir  = '/home/maspe/filer/SERT/' + ID + '/npys/'
figs_dir  = '/home/maspe/filer/SERT/' + ID + '/figs/'

In [ ]:
files_dir + 'entradas.xlsx'

#### Import info dictionary

In [4]:
#IDs = pickle.load(open('/home/maspe/filer/scripts/preprocessing/IDs.dict'))['dict']
baselines_all = pickle.load(open('/home/maspe/filer/SERT/ALL/npys/baselines.dict', 'rb'))
info = pickle.load(open(npys_dir + ID + '.info', 'rb'))

In [ ]:
info

## Main loop

### Epoching the continuos data

For baseline:<br>
Define a windows of $3 s$, beginning at each point from baselines dictionary. Points are at $30 Hz$, thus multiplied by $1000$ to yield a $30 KHz$ resolution.

For OF:<br>
Define windows of $3 s$. Create a windows equals to $3 \times fs$; $fs = 30 KHz$. Collect $3 s$ before and $3 s$ after each of the mouse's entrances to the center of the OF.

Save at '/home/maspe/filer/SERT/SERTXXXX/npys/mPFC_epochs'.


In [13]:
# Defining time windows as 3 seconds before plus 3 second after center entrances
fs = 30000
secs = 3
window = int(fs) * secs

n_baselines = 25
mPFC_nchannels = info['mPFC_nchannels']

for mouse in IDs:
    print('Processing mouse {}...'.format(mouse))
    # Read the entrances times
    df = pd.read_excel(files_dir + 'entradas.xlsx', sheet_name=0, header=None, names=["locs"])
    entrances_times = np.array(df['locs'].tolist(), dtype='int') * 30 # Times 30 because of sampling at 30 KHz
    n_epochs = len(entrances_times)
    print('Number of entrances: {}'.format(n_epochs))

    # Defining baselines as 3 s windows
    baselines = baselines_all[mouse] * 1000 # From 30 to 30,000 Hz
    
    print('Loading {}...'.format('mPFC'))
    mPFC = np.load(npys_dir + 'mPFC.npy')

    # For mPFC
    print('Collecting epochs for mPFC electrodes')   
    mPFC_epochs    = np.zeros((mPFC_nchannels, window * 2, n_epochs))
    mPFC_baselines = np.zeros((mPFC_nchannels, window, n_baselines))

    for channel in range(mPFC_nchannels):
        for epoch in range(n_epochs):
            mPFC_epochs[channel, :, epoch] = mPFC[channel, entrances_times[epoch] - window : entrances_times[epoch] + window]
            
        for baseline in range(n_baselines):
            mPFC_baselines[channel, :, baseline] = mPFC[channel, baselines[baseline] : baselines[baseline] + window]


print('Done!')

Processing mouse SERT1597...
Number of entrances: 16
Loading mPFC...
Done!


In [ ]:

        
# For NAC
print('Collecting epochs for NAC electrodes')
pre = []
post = []
NAC_epochs = np.zeros((NAC_nchannels, window * 2, n_epochs))

for channel in range(NAC_nchannels):
    for epoch in range(n_epochs):
        NAC_epochs[channel, :, epoch] = NAC[channel, entrances_times[epoch] - window : entrances_times[epoch] + window]
        
        
# For BLA
print('Collecting epochs for BLA electrodes')
pre = []
post = []
BLA_epochs = np.zeros((BLA_nchannels, window * 2, n_epochs))

for channel in range(BLA_nchannels):
    for epoch in range(n_epochs):
        BLA_epochs[channel, :, epoch] = BLA[channel, entrances_times[epoch] - window : entrances_times[epoch] + window]
        
        
# For vHip
print('Collecting epochs for vHip electrodes')
pre = []
post = []
vHip_epochs = np.zeros((vHip_nchannels, window * 2, n_epochs))

for channel in range(vHip_nchannels):
    for epoch in range(n_epochs):
        vHip_epochs[channel, :, epoch] = vHip[0, entrances_times[epoch] - window : entrances_times[epoch] + window]

#print('Saving epochs')
#np.save(npys_dir + 'mPFC_epochs', mPFC_epochs)
#np.save(npys_dir + 'NAC_epochs', NAC_epochs)
#np.save(npys_dir + 'BLA_epochs', BLA_epochs)
#np.save(npys_dir + 'vHip_epochs', vHip_epochs)
print('Done!')

### Plotting

In [ ]:
n_rows = np.int(np.ceil(n_epochs / 5.0))
### Plotting ###
# Plotting mPFC
print('Plotting mPFC...')
for channel in range(mPFC_nchannels):
    plt.figure(figsize=(20,10))
    for epoch in range(n_epochs):
        plt.subplot(n_rows, 5, epoch+1)
        plt.plot(mPFC_epochs[channel, :, epoch])
        
    plt.savefig(figs_dir + 'mPFC_ch' + str(channel) + '.png', dpi=150, format='png')
    plt.close()
    
print('mPFC: Done!')

# Plotting NAC
print('Plotting NAC...')
for channel in range(NAC_nchannels):
    plt.figure(figsize=(20,10))
    for epoch in range(n_epochs):
        plt.subplot(n_rows,5,epoch+1)
        plt.plot(NAC_epochs[channel,:, epoch])
        
    plt.savefig(figs_dir + 'NAC_ch' + str(channel) + '.png', dpi=150, format='png')
    plt.close()
    
print('NAC: Done!')
    

# Plotting BLA
print('Plotting BLA...')
for channel in range(BLA_nchannels):
    plt.figure(figsize=(20,10))
    for epoch in range(n_epochs):
        plt.subplot(n_rows,5,epoch+1)
        plt.plot(BLA_epochs[channel,:, epoch])
        
    plt.savefig(figs_dir + 'BLA_ch' + str(channel) + '.png', dpi=150, format='png')
    plt.close()
    
print('BLA: Done!')


# Plotting vHip
print('Plotting vHip...')
for channel in range(vHip_nchannels):
    plt.figure(figsize=(20,10))
    for epoch in range(n_epochs):
        plt.subplot(n_rows,5,epoch+1)
        plt.plot(vHip_epochs[channel,:, epoch])
        
    plt.savefig(figs_dir + 'vHip_ch' + str(channel) + '.png', dpi=150, format='png')
    plt.close()
    
print('vHip: Done!')
    
print('\n{} preprocessed!'.format(ID))

